In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
mapp = pd.read_csv('../data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl', drop=False)

pheno = pd.read_csv('../data/pheno_final.tsv', sep='\t')
pheno = pheno.drop_duplicates('subject')
pheno = pheno.set_index('subject', drop=False)

de   = pd.read_csv('../differential_expression_analysis/output/edgr_exclude_one/intersect.tsv', sep='\t')
de = de.set_index('ensembl', drop=False)

keep = pd.read_csv('../differential_expression_analysis/output/edgr_exclude_one/keep.intersect.txt', header=None)

pmapp = mapp[mapp.biotype == 'protein_coding'].copy()

keep = keep[keep[0].isin(pmapp.ensembl)].copy()
keep = pmapp.loc[keep[0], 'Description'].to_list()

de = de[de['ensembl'].isin(pmapp.ensembl)].copy().gene.to_list()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import os
network_files = os.listdir('../network_analysis/output')

def network_genes(genes):
    all_connectors = []
    i = 0
    for gene1 in genes:
#         if i % 100 == 0:
#             print('{}/{}'.format(i, len(genes)))

        i = i + 1
        
        if '{}_nearest_neighbor_weighted.txt'.format(gene1) not in network_files:
            continue
        net = pd.read_csv('output/{}_nearest_neighbor_weighted.txt'.format(gene1), sep='\t')
        net = net.set_index('End gene', drop=False)
        end_genes = net['End gene'].to_list()

        for gene2 in genes:
            if gene2 not in end_genes:
                continue
            connectors = net.loc[gene2, 'Connector genes']

            if (type(connectors) == float) or (type(connectors) == np.float64):
                continue

            all_connectors = all_connectors + connectors.split(',')

        all_connectors = list(set(all_connectors))
        
    return all_connectors

In [9]:
all_connectors = []
i = 0
for gene1 in de:
    if i % 100 == 0:
        print('{}/{}'.format(i, len(de)))

    if '{}_nearest_neighbor_weighted.txt'.format(gene1) not in network_files:
        continue
    net = pd.read_csv('../network_analysis/output/{}_nearest_neighbor_weighted.txt'.format(gene1), sep='\t')
    net = net.set_index('End gene', drop=False)
    end_genes = net['End gene'].to_list()

    for gene2 in de[:]:
        if gene2 not in end_genes:
            continue
        connectors = net.loc[gene2, 'Connector genes']

        if type(connectors) == float:
            continue

        all_connectors = all_connectors + connectors.split(',')
        
    all_connectors = list(set(all_connectors))
    
    i = i + 1


0/1171
100/1171
200/1171
300/1171
300/1171
400/1171
500/1171
600/1171
700/1171
800/1171
900/1171


In [10]:
len(all_connectors)

5083

In [13]:
with open('de_connectors.txt', 'w') as f:
    for s in all_connectors:
        f.write(s)
        f.write('\n')